In [ ]:
from matplotlib import pyplot as plt
from matplotlib.colors import ListedColormap
import pandas as pd
from sklearn.datasets import load_breast_cancer
import numpy as np
plt.style.use('seaborn-notebook')
%matplotlib notebook

In [ ]:
data_set = load_breast_cancer()
data = pd.DataFrame(data_set['data'], columns=data_set['feature_names'])
data['target'] = data_set['target']

In [ ]:
data

In [ ]:
class_names = {0: 'Malignant', 1: 'Benign'}

In [ ]:
variables = ['worst radius', 'mean texture']
X = data[variables].to_numpy()
y = data['target'].to_numpy()

In [ ]:
# Try a kNN classifier:
from sklearn.neighbors import KNeighborsClassifier
clf = KNeighborsClassifier(n_neighbors=1)
clf.fit(X, y)

In [ ]:
fig1, ax1 = plt.subplots(constrained_layout=True)
# Predict for many points to display the decision boundaries:
x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.1),
                     np.arange(y_min, y_max, 0.1))
Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)
colors = [
    [1., 0.49803922, 0.05490196],
    [0.12156863, 0.46666667, 0.70588235],
]
cmap = ListedColormap(colors)
ax1.contourf(xx, yy, Z, alpha=0.5, vmin=0, vmax=1, cmap=cmap)

class0 = data.loc[data['target'] == 0]
class1 = data.loc[data['target'] == 1]


for i, klass in enumerate((class0, class1)):
    print(class_names[i])
    s = ax1.scatter(
        klass[variables[0]].values,
        klass[variables[1]].values,
        s=100,
        label=class_names[i],
        color=colors[i]
    )
ax1.set_title(f'kNN with n_neighbors = {clf.n_neighbors}')
ax1.legend()
ax1.set_xlabel(variables[0]);
ax1.set_ylabel(variables[0]);

In [ ]:
# Let us check the performance via the confusion matrix:
from sklearn.model_selection import train_test_split
from sklearn.metrics import plot_confusion_matrix
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, stratify=y
)
clf = KNeighborsClassifier(n_neighbors=1)
clf.fit(X_train, y_train)
y_hat = clf.predict(X_test)
fig = plot_confusion_matrix(clf, X_test, y_test, display_labels=['Malignant', 'Benign'])
fig.ax_.set_title(f'kNN with n_neighbors = {clf.n_neighbors}')

In [ ]:
# Let us use cross-validation to find the best parameters:
from sklearn.model_selection import (
    GridSearchCV,
)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, stratify=y
)

clf = KNeighborsClassifier()
parameters = [{'n_neighbors': range(1, 11)}]
grid = GridSearchCV(
    clf,
    parameters,
    cv=5,
    scoring='precision',
    return_train_score=True,
)
grid.fit(X_train, y_train)
print('Best parameters for knn:', grid.best_params_)

In [ ]:
fig1, ax1 = plt.subplots(constrained_layout=True)
ax1.errorbar(
    parameters[0]['n_neighbors'],
    grid.cv_results_['mean_test_score'],
    yerr=grid.cv_results_['std_test_score'],
    marker='o', markersize=14
)
ax1.set(xlabel='n_neighbors', ylabel=grid.scoring)
ax1.set_title('Optimizing n_neighbors for a k-nearest neighbors classifier');